In [1]:
import numpy as np, pandas as pd

In [ ]:
def logistic(x, theta):
    return 1.0 / ( 1 + np.exp(-x * theta) )

def logisticError(x, theta, y):
    yhat = logistic(x, theta)
    err = -(y.T*np.log(yhat) + (1-y).T*np.log(1-yhat)) / len(y)
    return float(err), yhat
    
def updateTheta(a, yhat, y, x, th, errDiff):
    d = ((yhat-y).T * x).T / len(y)
    return th - (a*d)

def makeIndepDepData(data):
    y = data[:, -1] # actual
    x = np.hstack((np.ones([data.shape[0],1]), data[:, :-1])) # put intercept
    return x,y

In [ ]:
def learn_model( data, alpha, eps, verbose=False):
    x,y = makeIndepDepData(data)
    theta = np.matrix(np.random.rand(x.shape[1],1)*2 - 1) # initialize Theta
    lastErr = 0
    err,yhat = logisticError(x, theta, y)

    c = 0
    while (abs(err-lastErr) > eps) and c<1e6:
        if c % 1000 == 0 and verbose:
            print('Iter #%u, error: %f'%(c,err))
        theta = updateTheta(alpha, yhat, y, x, theta, err-lastErr)
        lastErr = err
        err,yhat = logisticError(x, theta, y)
        if err > lastErr:
            alpha /= 10
        c += 1
    
    print('Final iteration #%u, error: %f' % (c-1,err) )
    return theta


train_data = generate_data( clean_data, 100, "hills")
model = learn_model( train_data, 0.1, 1e-7, True)
print('Coefficients: %s' % str(model.T))